# Set Up

In [1]:
import pandas as pd

import nltk
#nltk.download('all')
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

from Util import *

In [73]:
# Load Data
df = pd.read_csv("/content/Train_Data.csv")

In [74]:
df.head()

,ID,Number,Description,Keywords
0,699,71988,A man in a wheelchair and another sitting on a...,"['man', 'wheelchair', 'another', 'bench', 'wat..."
1,701,193622,A man sits with a traditionally decorated cow,"['man', 'traditionally decorated cow']"
2,827,52087,A man getting a drink from a water fountain th...,"['man', 'drink', 'water fountain', 'toilet']"
3,891,119964,A person holding a skateboard overlooks a dead...,"['person', 'skateboard', 'dead field of crops']"
4,1221,382406,A woman is walking a dog in the city.,"['woman', 'dog', 'city']"


In [75]:
df['BIO_tags'] = df['Description'].apply(DataAnnotate)

In [76]:
df.head()

,ID,Number,Description,Keywords,BIO_tags
0,699,71988,A man in a wheelchair and another sitting on a...,"['man', 'wheelchair', 'another', 'bench', 'wat...","[B-DT, B-NN, B-IN, B-DT, B-NN, B-CC, B-DT, B-V..."
1,701,193622,A man sits with a traditionally decorated cow,"['man', 'traditionally decorated cow']","[B-DT, B-NN, B-VBZ, B-IN, B-DT, B-RB, B-VBN, B..."
2,827,52087,A man getting a drink from a water fountain th...,"['man', 'drink', 'water fountain', 'toilet']","[B-DT, B-NN, B-VBG, B-DT, B-NN, B-IN, B-DT, B-..."
3,891,119964,A person holding a skateboard overlooks a dead...,"['person', 'skateboard', 'dead field of crops']","[B-DT, B-NN, B-VBG, B-DT, B-NN, B-VBZ, B-DT, B..."
4,1221,382406,A woman is walking a dog in the city.,"['woman', 'dog', 'city']","[B-DT, B-NN, B-VBZ, B-VBG, B-DT, B-NN, B-IN, B..."


In [77]:
df.to_csv("IOB_Annotated_Data.csv",index=False)

In [78]:
processed_df = Data_Mapping(df,'Description','Keywords')

In [79]:
processed_df['Predicted_Class'] = Data_Mapping_iob(df,'Description','BIO_tags','Keywords')['Class']
processed_df.shape

(642982, 4)

In [ ]:
processed_df['Predicted_Class'].fillna("O")

In [80]:
processed_df

,ID,Token,Class,Predicted_Class
0,699,A,O,O
1,699,man,Noun,NOUN
2,699,in,O,O
3,699,a,O,O
4,699,wheelchair,Noun,NOUN
...,...,...,...,...
642977,825532,of,O,NaN
642978,825532,a,O,NaN
642979,825532,clock,O,NaN
642980,825532,tower,O,NaN


In [81]:
# Define mapping dictionary
label_map = {'Noun': 1, 'O': 0}

# Map values in 'Labels' column
processed_df['Class'] = processed_df['Class'].map(label_map)
processed_df['Predicted_Class'] = processed_df['Predicted_Class'].map(label_map)

In [82]:
processed_df['Predicted_Class']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
642977    0.0
642978    0.0
642979    0.0
642980    0.0
642981    0.0
Name: Predicted_Class, Length: 642982, dtype: float64

In [84]:
processed_df.isnull().sum()

ID                      0
Token                   0
Class                   0
Predicted_Class    254754
dtype: int64

In [60]:
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score
#accuracy, precision, recall and F1

metrics_df = pd.DataFrame()

# Compute accuracy,precision, recall, and F1 score
accuracy = accuracy_score(processed_df['Class'], processed_df['Predicted_Class'])
precision = precision_score(processed_df['Class'], processed_df['Predicted_Class'])
recall = recall_score(processed_df['Class'], processed_df['Predicted_Class'])
f1 = f1_score(processed_df['Class'], processed_df['Predicted_Class'])

# Print the results
print("Accuracy:", accuracy*100)
print("Precision:", precision*100)
print("Recall:", recall*100)
print("F1 Score:", f1*100)

pred_noun_count = processed_df[processed_df['Predicted_Class']==1].shape[0]
tokens_noun_count = processed_df[processed_df['Class']==1].shape[0]
covered_area = pred_noun_count/tokens_noun_count
print("the percentage of covered area:", covered_area)
from collections import defaultdict
results = defaultdict(list)
results['Model'].append("BERT_Baseline")
results['Accuracy'].append(round(accuracy*100,2))
results['Precision'].append(round(precision*100,2))
results['Recall'].append(round(recall*100,2))
results['F1 Score'].append(round(f1*100,2))
results['Covered Area'].append(round(covered_area*100,2))

metrics_df = metrics_df.append(results, ignore_index=True)

metrics_df


ValueError: Input y_pred contains NaN.